# Why can't we use CNN for this purpose, as it is a supervised approach?
The authors tried this but they faced the below issue,
- Hand engineering the loss function doesn't work well.

What if the loss function is to 'Create images that are indistinguishable from reality'. This is exactly what GANs do. Therefore, GANs is used inorder to learn the loss function suitable for the task.

**GENERATOR**
- Inspired by U-NET

**DISCRIMINATOR** (PatchGAN)
- Couple of Conv Layers

Output is a grid image where each patch gives values between (0,1) indicating whether it is real or fake



In [1]:
import torch
import torch.nn as nn

## Discriminator & Generator

In [2]:
from torch.nn.modules.conv import ConvTranspose2d
class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=2):
    super(CNNBlock, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            4,
            stride,
            bias=False,
            padding_mode='reflect'
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )

  def forward(self, x):
    return self.conv(x)

# x, y -> concatenate these along the channels
class Discriminator(nn.Module):
  def __init__(self, in_channels=3, features=[64, 128, 256, 512]): # 256 image -> 26x26
    super(Discriminator, self).__init__()
    self.initial = nn.Sequential(
        nn.Conv2d(
          in_channels*2,
          features[0],
          kernel_size=4,
          stride=2,
          padding=1,
          padding_mode='reflect'
        ),
        nn.LeakyReLU(0.2)
    )
    
    layers = []
    in_channels = features[0]
    for feature in features[1:]:
      layers.append(
          CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2)
      )
      in_channels = feature
    layers.append(
        nn.Conv2d(
            in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode='reflect'
        )
    )
    self.model = nn.Sequential(*layers)
  
  def forward(self, x, y):
    x = torch.cat([x, y], dim=1)
    x = self.initial(x)
    # the only difference between the init & cnnblock is that 
    # we have used bias instead of BN layer in the init block
    return self.model(x)

# def test():
#   x = torch.randn((1, 3, 256, 256))
#   y = torch.randn((1, 3, 256, 256))
#   model = Discriminator()
#   preds = model(x, y)
#   print(preds.shape)
# test()

class Block(nn.Module):
  def __init__(self, in_channels, out_channels, down=True, act='relu', use_dropout=False):
    super(Block, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode='reflect')
        if down
        else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU() if act=='relu' else nn.LeakyReLU(0.2),
    )

    self.use_dropout = use_dropout
    self.dropout = nn.Dropout(0.5)
  
  def forward(self, x):
    x = self.conv(x)
    return self.dropout(x) if self.use_dropout else x

class Generator(nn.Module):
  def __init__(self, in_channels=3, features=64):
    super(Generator, self).__init__()
    self.initial_down = nn.Sequential(
        nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode='reflect'),
        nn.LeakyReLU(0.2),
    ) # 128

    self.down1 = Block(features, features*2, down=True, act='leaky', use_dropout=False) # 64
    self.down2 = Block(features*2, features*4, down=True, act='leaky', use_dropout=False) # 32
    self.down3 = Block(features*4, features*8, down=True, act='leaky', use_dropout=False) # 16
    self.down4 = Block(features*8, features*8, down=True, act='leaky', use_dropout=False) # 8
    self.down5 = Block(features*8, features*8, down=True, act='leaky', use_dropout=False) # 4
    self.down6 = Block(features*8, features*8, down=True, act='leaky', use_dropout=False) # 2

    self.bottleneck = nn.Sequential(
        nn.Conv2d(features*8, features*8, 4, 2, 1, padding_mode='reflect'),
        nn.ReLU(),
    ) # 1x1

    self.up1 = Block(features*8, features*8, down=False, act='relu', use_dropout=True)
    self.up2 = Block(features*8*2, features*8, down=False, act='relu', use_dropout=True)
    self.up3 = Block(features*8*2, features*8, down=False, act='relu', use_dropout=True)
    self.up4 = Block(features*8*2, features*8, down=False, act='relu', use_dropout=False)
    self.up5 = Block(features*8*2, features*4, down=False, act='relu', use_dropout=False)
    self.up6 = Block(features*4*2, features*2, down=False, act='relu', use_dropout=False)
    self.up7 = Block(features*2*2, features, down=False, act='relu', use_dropout=False)

    self.final_up = nn.Sequential(
        nn.ConvTranspose2d(features*2, in_channels, kernel_size=4, stride=2, padding=1),
        nn.Tanh(),
    )


  def forward(self, x):
    d1 = self.initial_down(x)
    d2 = self.down1(d1)
    d3 = self.down2(d2)
    d4 = self.down3(d3)
    d5 = self.down4(d4)
    d6 = self.down5(d5)
    d7 = self.down6(d6)
    bottleneck = self.bottleneck(d7)
    u1 = self.up1(bottleneck)
    u1 = torch.cat([u1, d7], 1)
    u2 = self.up2(u1)
    u2 = torch.cat([u2, d6], 1)
    u3 = self.up3(u2)
    u3 = torch.cat([u3, d5], 1)
    u4 = self.up4(u3)
    u4 = torch.cat([u4, d4], 1)
    u5 = self.up5(u4)
    u5 = torch.cat([u5, d3], 1)
    u6 = self.up6(u5)
    u6 = torch.cat([u6, d2], 1)
    u7 = self.up7(u6)
    u7 = torch.cat([u7, d1], 1)

    return self.final_up(u7)

# def test():
#   x = torch.randn((1, 3, 256, 256))
#   model = Generator()
#   preds = model(x)
#   print(preds.shape)
# test()


## Config

In [13]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
LEARNING_RATE = 2e-4
BATCH_SIZE = 16
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 500
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_DISC = "/content/drive/MyDrive/Tutorials/disc.pth.tar"
CHECKPOINT_GEN = "/content/drive/MyDrive/Tutorials/gen.pth.tar"

both_transform = A.Compose(
    [A.Resize(width=256, height=256),], additional_targets={"image0": "image"},
)

transform_only_input = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(p=0.2),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

## Utils

In [14]:
import torch
from torchvision.utils import save_image

def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization #
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        # if epoch == 1:
        save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

## Dataset

In [ ]:
# !pip install -q kaggle
# !mkdir ~/.kaggle
# !cp /content/kaggle.json ~/.kaggle
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d vikramtiwari/pix2pix-dataset
# !unzip /content/pix2pix-dataset.zip

In [15]:
from PIL import Image
import numpy as np
import os
from torch.utils.data import Dataset

In [16]:
class MapDataset(Dataset):
  def __init__(self, root_dir):
    super(MapDataset, self).__init__()
    self.root_dir = root_dir
    self.list_files = os.listdir(self.root_dir)

  def __len__(self):
    return len(self.list_files)

  def __getitem__(self, index):
    img_file = self.list_files[index]
    img_path = os.path.join(self.root_dir, img_file)
    image = np.array(Image.open(img_path))
    # the image is 600x600 & the map dataset has input & target
    # stitched side by side
    input_image = image[:, :600, :]
    target_image = image[:, 600:, :]

    augmentations = both_transform(image=input_image, image0=target_image)
    input_image, target_image = augmentations['image'], augmentations['image0']

    input_image = transform_only_input(image=input_image)['image']
    target_image = transform_only_mask(image=target_image)['image']

    return input_image, target_image
  

## Training

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.utils import save_image

In [18]:
def train_fn(disc, gen, loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler):
  loop = tqdm(loader, leave=True)

  for idx, (x, y) in enumerate (loop):
    x, y, = x.to(DEVICE), y.to(DEVICE)

    # Train Discriminator
    with torch.cuda.amp.autocast():
      y_fake = gen(x)
      D_real = disc(x, y)
      D_fake = disc(x, y_fake.detach())
      D_real_loss = BCE(D_real, torch.ones_like(D_real))
      D_fake_loss = BCE(D_fake, torch.zeros_like(D_fake))
      D_loss = (D_real_loss + D_fake_loss) / 2

    disc.zero_grad()
    d_scaler.scale(D_loss).backward()
    d_scaler.step(opt_disc)
    d_scaler.update()

    # Train Generator
    with torch.cuda.amp.autocast():
      D_fake = disc(x, y_fake)
      G_fake_loss = BCE(D_fake, torch.ones_like(D_fake))
      L1 = L1_LOSS(y_fake, y) * L1_LAMBDA
      G_loss = (G_fake_loss + L1)

    gen.zero_grad()
    g_scaler.scale(G_loss).backward()
    g_scaler.step(opt_gen)
    g_scaler.update()


def main():
  disc = Discriminator(in_channels=3).to(DEVICE)
  gen = Generator(in_channels=3).to(DEVICE)
  opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
  opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
  BCE = nn.BCEWithLogitsLoss()
  L1_LOSS = nn.L1Loss()

  if LOAD_MODEL:
    load_checkpoint(CHECKPOINT_GEN, gen, opt_gen, LEARNING_RATE)
    load_checkpoint(CHECKPOINT_DISC, disc, opt_disc, LEARNING_RATE)

  train_dataset = MapDataset(root_dir='data/maps/train')
  train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
  g_scaler = torch.cuda.amp.GradScaler() # for fp16 training 
  d_scaler = torch.cuda.amp.GradScaler()
  val_dataset = MapDataset(root_dir='data/maps/val')
  val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle=False)

  for epoch in range(NUM_EPOCHS):
    train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler)

    if SAVE_MODEL and epoch % 5 == 0:
      save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
      save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)
    
    # create an evaluation folder in the root directory
      save_some_examples(gen, val_loader, epoch, folder='evaluation')
  



if __name__ == '__main__':
  main()

!zip -r /content/evaluation.zip /content/evaluation
!cp /content/evaluation.zip /content/drive/MyDrive/Tutorials
# !cp /content/disc.pth.tar /content/drive/MyDrive/Tutorials
# !cp /content/gen.pth.tar /content/drive/MyDrive/Tutorials

=> Loading checkpoint
=> Loading checkpoint


100%|██████████| 69/69 [00:26<00:00,  2.57it/s]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 69/69 [00:18<00:00,  3.68it/s]


=> Saving checkpoint
=> Saving checkpoint


...

=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 69/69 [00:18<00:00,  3.64it/s]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 69/69 [00:20<00:00,  3.35it/s]


  adding: content/evaluation/ (stored 0%)
  adding: content/evaluation/input_270.png (deflated 0%)
  adding: content/evaluation/input_55.png (deflated 0%)
  adding: content/evaluation/label_265.png (deflated 0%)
  adding: content/evaluation/y_gen_155.png (deflated 0%)
  adding: content/evaluation/input_125.png (deflated 0%)
  adding: content/evaluation/label_25.png (deflated 0%)
  adding: content/evaluation/label_345.png (deflated 0%)
  adding: content/evaluation/y_gen_405.png (deflated 0%)
  adding: content/evaluation/input_90.png (deflated 0%)
  adding: content/evaluation/input_40.png (deflated 0%)
  adding: content/evaluation/y_gen_415.png (deflated 0%)
  adding: content/evaluation/input_70.png (deflated 0%)
  adding: content/evaluation/input_490.png (deflated 0%)
  adding: content/evaluation/input_265.png (deflated 0%)
  adding: content/evaluation/label_335.png (deflated 0%)
  adding: content/evaluation/label_390.png (deflated 0%)
  adding: content/evaluation/y_gen_350.png (deflate